In [81]:
from openai import OpenAI
import pandas as pd
from tqdm import tqdm
import numpy as np
import random
import os

In [82]:
client = OpenAI(api_key="sk-5659e4563ce54759b130c0fa617e288d", base_url="https://api.deepseek.com")

In [83]:
codes = pd.read_csv("./results3/7-3/3-code-10_2.csv",header=None)

In [84]:
descs = pd.read_csv("./results3/7-3/3-code-10_2_desc.csv",header=None)

In [85]:
bscores = pd.read_csv("./results3/7-3/3-code-10_2_bscore.csv",header=None)

In [86]:
bseveritys = pd.read_csv("./results3/7-3/3-code-10_2_bseverity.csv",header=None)

In [87]:
codes1 = codes[0]
codes2 = codes[1]
codes3 = codes[2]
codes4 = codes[3]
codes5 = codes[4]

In [88]:
descs1 = descs[0]
descs2 = descs[1]
descs3 = descs[2]
descs4 = descs[3]
descs5 = descs[4]

In [89]:
bscores1 = bscores[0]
bscores2 = bscores[1]
bscores3 = bscores[2]
bscores4 = bscores[3]
bscores5 = bscores[4]

In [90]:
bseveritys1 = bseveritys[0]
bseveritys2 = bseveritys[1]
bseveritys3 = bseveritys[2]
bseveritys4 = bseveritys[3]
bseveritys5 = bseveritys[4]

In [91]:
df = pd.read_csv('./dataset/test/test_code.csv', header=None)

In [92]:
test_codes = df[0].tolist()

In [93]:
df = pd.read_csv('./dataset/test/test_desc.csv', header=None)

In [94]:
test_descs = df[0].tolist()

In [95]:
import tiktoken
encoder = tiktoken.get_encoding("cl100k_base")
def cut_text_to_fit_token_limit(text, max_tokens):
    tokens = encoder.encode(text)
    if len(tokens) > max_tokens:
        tokens = tokens[:max_tokens]
        truncated_text = encoder.decode(tokens)
        return truncated_text
    return text

In [96]:
def get_token_count(text):
    """
    计算输入文本的token数量

    参数:
    text (str): 输入的文本

    返回:
    int: 文本的token数量
    """
    # 使用编码器计算tokens
    tokens = encoder.encode(text)
    num_tokens = len(tokens)

    return num_tokens

In [97]:
max_tokens = 32600
# 32768
n = 5
per_tokens = max_tokens//n+1
for i in range(len(codes1)):
    codes[0][i] = cut_text_to_fit_token_limit(codes[0][i] ,per_tokens)
    codes[1][i] = cut_text_to_fit_token_limit(codes[1][i] ,per_tokens)
    codes[2][i] = cut_text_to_fit_token_limit(codes[2][i] ,per_tokens)
    codes[3][i] = cut_text_to_fit_token_limit(codes[3][i] ,per_tokens)
    codes[4][i] = cut_text_to_fit_token_limit(codes[4][i] ,per_tokens)

    descs[0][i] = cut_text_to_fit_token_limit(descs[0][i] ,per_tokens)
    descs[1][i] = cut_text_to_fit_token_limit(descs[1][i] ,per_tokens)
    descs[2][i] = cut_text_to_fit_token_limit(descs[2][i] ,per_tokens)
    descs[3][i] = cut_text_to_fit_token_limit(descs[3][i] ,per_tokens)
    descs[4][i] = cut_text_to_fit_token_limit(descs[4][i] ,per_tokens)

In [98]:
demo_ins = "I will give you some demo examples that include the vulnerability code and a description of the vulnerability, as well as the severity level of the vulnerability. At the end of the demo, I will give you a test example that includes the vulnerability code and a description of the vulnerability. You only need to output the severity level of the vulnerability of the test example code and description, without outputting the corresponding explanation, or the vulnerability level of the previous demo examples.\n"
demo1 = "Demo 1:\n"
demo2 = "Demo 2:\n"
demo3 = "Demo 3:\n"
demo4 = "Demo 4:\n"
demo5 = "Demo 5:\n"
demos = []
test = "Test 1:\n"
demos.append(demo1)
demos.append(demo2)
demos.append(demo3)
demos.append(demo4)
demos.append(demo5)

instruct = "Determine Base Severity of the vulnerability code based on the following vulnerable code (a function) and the textual description of the vulnerability.\n"
# the Base Score and
input_insturction = ' [input] '
output_insturction = ' [output] '

In [99]:
def comment(prompt):
#     nn = len(code)
#     prompt = ""
#     for i in range(nn):
#         prompt += instruct
#         prompt += input_insturction
#         prompt += "Code: "
#         prompt += code[i]
#         prompt += "\n"
#         prompt += "Description: "
#         prompt += desc[i]
#         prompt += "\n"
#         prompt += output_insturction
#         if i < nn-1:
# #             prompt += "Base Score:"
# #             prompt += Base_Score[i]
# #             prompt += "\n"
#             prompt += "Base Severity:"
#             prompt += Base_Severity[i]
#             prompt += "\n\n"

    # prompt = 'Here is an incomplete title for a Stack Overflow post:\n' + incomplete_title + \
    #          '\nHere is a description of the post:\n' + description + \
    #          '\nHere is the code for this post:\n' + code + \
    #          '\nHere is the final title(you just need to output the content after the incomplete title, without generating the incomplete title section in front):\n'

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "You are an expert in code vulnerability assessment, and you will rate the vulnerabilities based on the following scoring criteria:\n"+\
                                          "0.1-3.9:LOW, 4.0–6.9:Medium, 7.0–8.9:High, 9.0–10.0:Critical"},
            {"role": "user", "content": prompt},
        ],
        temperature=0
    )

    return response.choices[0].message.content

In [100]:
import time
import math
import random

In [101]:
import sys
fail = []
have_money = 1
num_batches = math.ceil(len(test_codes) / 100)
for batch_index in range(num_batches):
    start_index = batch_index * 100
    end_index = min(start_index + 100, len(test_codes))
    print(start_index)
    # codes_batch = test_codes[start_index:end_index]
    # descs_batch = test_descs[start_index:end_index]
    for i in tqdm(range(start_index, end_index)):
        ans = []
        pro = []

        idx = [0,1,2,3]
        random.shuffle(idx)

        para_codes = []
        para_codes.append(codes[idx[0]][i])
        para_codes.append(codes[idx[1]][i])
        para_codes.append(codes[idx[2]][i])
        para_codes.append(codes[idx[3]][i])
        para_codes.append(test_codes[i])

        para_descs = []
        para_descs.append(descs[idx[0]][i])
        para_descs.append(descs[idx[1]][i])
        para_descs.append(descs[idx[2]][i])
        para_descs.append(descs[idx[3]][i])
        para_descs.append(test_descs[i])

        para_bscores = []
        para_bscores.append(bscores[idx[0]][i])
        para_bscores.append(bscores[idx[1]][i])
        para_bscores.append(bscores[idx[2]][i])
        para_bscores.append(bscores[idx[3]][i])

        para_bseveritys = []
        para_bseveritys.append(bseveritys[idx[0]][i])
        para_bseveritys.append(bseveritys[idx[1]][i])
        para_bseveritys.append(bseveritys[idx[2]][i])
        para_bseveritys.append(bseveritys[idx[3]][i])

        success = 0
        fail_count = 0

        nn = len(para_codes)
        prompt = ""
        prompt += demo_ins
        for j in range(nn):
            if j < nn-1:
                prompt += demos[j]
            else:
                prompt += test
            prompt += instruct
            prompt += input_insturction
            prompt += "Code: "
            prompt += para_codes[j]
            prompt += "\n"
            prompt += "Description: "
            prompt += para_descs[j]
            prompt += "\n"
            prompt += output_insturction
            if j < nn-1:
    #             prompt += "Base Score:"
    #             prompt += Base_Score[i]
    #             prompt += "\n"
                prompt += "Base Severity:"
                prompt += para_bseveritys[j]
                prompt += "\n\n"
        pro.append(prompt)
        while success!=1:
            try:
                ans.append(comment(prompt))
                success = 1
                time.sleep(2)
            except Exception as e:
                info = e.args[0]
                print("Error: ", info)
                if str(info).find("402") != -1:
                    print("402 Error encountered. Exiting program.")
                    sys.exit(1)
                time.sleep(2)
                fail_count += 1
            if fail_count>5:
                fail.append(i)
                ans.append("ERROR")
                break
        # time.sleep(5)
        if have_money == 0:
            break
        if get_token_count(pro[0]) > max_tokens:
            print("-----",i,"-----")
        df = pd.DataFrame(ans)
        ddf = pd.DataFrame(pro)
        if batch_index == 0 and i == 0:
            df.to_csv('./results_RQ4/results_RQ4_deepseek/Rand/answers.csv', index=False, header=None, lineterminator='\n')
            ddf.to_csv('./results_RQ4/results_RQ4_deepseek/Rand/prompts.csv', index=False, header=None, lineterminator='\n')
        else:
            with open('./results_RQ4/results_RQ4_deepseek/Rand/answers.csv', 'a', encoding='utf-8') as f:
                df.to_csv(f, index=False, header=None, lineterminator='\n')
            with open('./results_RQ4/results_RQ4_deepseek/Rand/prompts.csv', 'a', encoding='utf-8') as f:
                ddf.to_csv(f, index=False, header=None, lineterminator='\n')
    if have_money == 0:
        break
ddd = pd.DataFrame(fail)
ddd.to_csv('./results_RQ4/results_RQ4_deepseek/Rand/fail.csv', index=False, header=None, lineterminator='\n')

0


100%|██████████| 100/100 [06:12<00:00,  3.72s/it]


100


 29%|██▉       | 29/100 [01:45<04:38,  3.93s/it]

Error:  Error code: 400 - {'detail': 'Content Exists Risk'}
Error:  Error code: 400 - {'detail': 'Content Exists Risk'}
Error:  Error code: 400 - {'detail': 'Content Exists Risk'}
Error:  Error code: 400 - {'detail': 'Content Exists Risk'}
Error:  Error code: 400 - {'detail': 'Content Exists Risk'}
Error:  Error code: 400 - {'detail': 'Content Exists Risk'}


...

 40%|████      | 40/100 [02:30<04:05,  4.09s/it]

Error:  Error code: 400 - {'detail': 'Content Exists Risk'}
Error:  Error code: 400 - {'detail': 'Content Exists Risk'}
Error:  Error code: 400 - {'detail': 'Content Exists Risk'}
Error:  Error code: 400 - {'detail': 'Content Exists Risk'}
Error:  Error code: 400 - {'detail': 'Content Exists Risk'}
Error:  Error code: 400 - {'detail': 'Content Exists Risk'}


100%|██████████| 100/100 [07:08<00:00,  4.29s/it]


1200


100%|██████████| 8/8 [00:38<00:00,  4.86s/it]


In [ ]:
data1 = pd.read_csv("./results_RQ4/results_RQ4_deepseek/Rand/prompts.csv",header=None)